In [5]:
import pandas as pd
import numpy as np
import pdfplumber
import re

- how the sub-products are positioneted in the pdf for each products
- creating class for each of the main product
- doing sama operation for each of the page talking about the same product
- creating function with 1st params(how they divide pdf in the first way) 2nd, 3rd ecc

**make sense to have a pdf creator after the regex operation? Make pdf table after operating with the regex**

# extracting text from pdf, from certain page

In [163]:
ctt = pdfplumber.open('ctt.pdf')
page = ctt.pages[4]
text_ctt = page.extract_text()
abanca = pdfplumber.open('abanca1.pdf')
pagea = abanca.pages[4]
text_abanca = pagea.extract_text()
table = pagea.extract_table()
pd.DataFrame(table).iloc[10,0]

'3.   Manutenção de conta \n3.1 Conta D.O. Particulares (Nota 2) \n- Saldo médio até 1.000€ \n- Saldo médio entre 1.001€ e 2.500€ \n- Saldo médio superior a 2.500€ \n3.2 Conta Ordenado (Nota 1 e Nota 16) \n3.3 Conta Standard (não admite novas contratações) \n3.4 Conta Future (Nota 3) \n3.5 Conta Kids (Nota 4) \n3.6 Conta Global (Nota 5) \n3.7 Conta Base (Nota 6) \n3.8 Conta Private (Nota 7)  \n3.9 Conta Value (Nota 8) \n3.10 Conta Smart (Nota 9) \n3.11 Conta Futuro (Nota 10) \n3.12 Conta Serviços Mínimos Bancários (Nota 11) \n3.13 Conta Moeda Estrangeira (Nota 12) \n3.14 Conta ABANCA Internacional (Nota 17)'

## Cleaning texts

In [122]:
text_abanca=text_abanca.lower()
text_abanca = re.sub('isento', '0,00', text_abanca)
text_ctt=text_ctt.lower()
text_ctt = re.sub('isento', '0,00', text_ctt)

In [159]:
abanca = pdfplumber.open('bankinter_pricelists.pdf')
pagea = abanca.pages[8]
table = pagea.extract_table()
pd.DataFrame(table)

,0,1,2,3,4,5,6,7,8,9,10,11,12
0,None,None,Comissões,None,None,Acresce Imposto,None,None,None,Comissões,None,None,
1,None,None,Valores Sem Imposto,None,None,,IVA,None,None,Valor Com Imposto,None,None,
2,None,None,Em %,Euros \n(min/máx),Valor \nAnual,IS,Cont.,Mad.,Aç.,Cont.,Mad.,Aç.,Outras Condições
3,Contas à Ordem,None,None,None,None,None,None,None,None,None,None,None,None
4,Conta Mais Ordenado,None,None,None,None,None,None,None,None,None,None,None,None
5,1.0,Comissão de Manutenção de Conta,--,Isento,--,--,--,--,--,Isento,None,None,
6,Conta Bankinter (idade igual ou superior a 18 ...,None,None,None,None,None,None,None,None,None,None,None,None
7,1.0,Comissão de Manutenção de Conta,--,"15,00 \n(trimestral)","60,00",4%,--,--,--,"15,60 (trimestral)",None,None,Ver Nota (1).
8,- Conta Bankinter em Moeda Estrangeira. \n- Co...,None,None,None,None,None,None,None,None,None,None,None,None
9,1.0,Comissão de Manutenção de Conta,--,"20,00 \n(trimestral)","80,00",4%,--,--,--,"20,80 (trimestral)",None,None,Ver Nota (2).


- **DEFINITION**: 

In [144]:
def extract_clean(pdf,n_page):
    file = pdfplumber.open(pdf)
    page = file.pages[n_page]
    text = page.extract_text().lower()
    text = re.sub('isento', '0,00', text)
    return text    

In [145]:
bankinter = extract_clean('bankinter_pricelists.pdf', 8)

In [153]:
# bankinter

## getting different subsection in 1.1

In [140]:
#info of all the info in tab 1.1 'depositos a ordem' ctt
regex = '[0-9]{1}\.\D'
sections_ctt = re.split(regex,text_ctt)
#info of all the info in tab 1.1 'depositos a ordem' abanca
regex = '[0-9]{1}\.\D'
sections_abanca = re.split(regex,text_abanca)

In [154]:
# sections_bank = re.split(regex,bankinter)
# contas_bank = []
# pattern = 'manutenção de conta'
# for section in sections_bank:
#     section = section.lower()
#     if pattern in section:
#         contas_bank.append(section)
# contas_bank

- **DEFINITION**

In [155]:
def subsection(text):
    regex = '[0-9]{1}\.\D'
    sections = re.split(regex,text)
    return text

# Manutencao do conta

- problem with bankinter: they have a double line for the price list (before the number and then the trimestral so the value appears before the 'manutencao de conta'

## for bank with info for each account in section (ctt f.e.)

In [157]:
# get string for all different account in mantaining 
pattern = 'manutenção de conta'
contas_ctt = []
for section in sections_ctt:
    section = section.lower()
    if pattern in section:
        contas_ctt.append(section)
        
#get n. of product for bank
n_accounts = len(contas_ctt)
contas_ctt

['omissão de manutenção de conta isento isento is - 4%\n',
 'omissão de manutenção de conta isento isento --\n',
 'omissão de manutenção de conta 4,50 54,00 is - 4% nota (6)\n',
 'omissão de manutenção de conta isento isento -- nota (7)\n']

- **DEFINITION**

In [156]:
def manutencao_plus(sections):
    pattern = 'manutenção de conta'
    contas= []
    for section in sections:
        if pattern in section:
            contas.append(section)
    n_accounts = len(contas)
    return (contas,n_accounts)

## for bank that has all the info in one single section (abanca f.e.)

In [128]:
# get string for section corrisponding to mantainig of account
pattern = 'manutenção de conta'
manutencao =''
for section in sections_abanca:
    if pattern in section:
        manutencao=''.join(section)
contas_abanca = re.split('[0-9]{1}\.[0-9]{1,2}\D',manutencao)
n_products = len(contas_abanca)-1 #-1 cause of the title that count as element of the list
contas_abanca

['  manutenção de conta     \n',
 'conta d.o. particulares (nota 2)     \n- saldo médio até 1.000€  15,00 trimestral  60,00 \n- saldo médio entre 1.001€ e 2.500€  10,00 trimestral  40,00 \n- saldo médio superior a 2.500€  0,00  n/a \n',
 'conta ordenado (nota 1 e nota 16)  0,00  n/a \n',
 'conta standard (não admite novas contratações)  0,00  n/a \n',
 'conta future (nota 3)  0,00  n/a \n',
 'conta kids (nota 4)  0,00  n/a \nis – 4%   \n',
 'conta global (nota 5)  6,53 / 7,23 mensal  85,14 \n',
 'conta base (nota 6)  10,00 mensal  120,00  \n',
 'conta private (nota 7)   8,00 mensal  96,00 \n',
 'conta value (nota 8)  5,00 mensal  60,00 \n',
 'conta smart (nota 9)  5,00 mensal  60,00 \n',
 'conta futuro (nota 10)  0,00  n/a \n',
 'conta serviços mínimos bancários (nota 11)  2,00 semestral  4,00 \n',
 'conta moeda estrangeira (nota 12)  5,00 mensal  60,00 \n',
 'conta abanca internacional (nota 17)  0,00  n/a \n']

In [126]:
def manutencao_1(sections):
    pattern = 'manutenção de conta'
    manutencao =''
    for section in sections:
        if pattern in section:
            manutencao=''.join(section)
    contas = re.split('[0-9]{1}\.[0-9]{1,2}\D',manutencao)
    n_products = len(contas)-1
    return (contas,n_products)

14

# Extrato

In [127]:
pattern = 'extrato'
extracto_abanca = ''
for section in sections_abanca:
    section = section.lower()
    if pattern in section:
        extracto_abanca = ''.join(section)
extracto_abanca        

' emissão de extrato     \n1.1  mensal (enviado ao domicílio)  0,00 \nn/a  iva – 23% \n1.2  outros, para além do indicado em 1.1  2,00 por extrato \n1.3  2ª via      7,50   \n'